In [ ]:
#
# CONFIGURE RUN-TIME PARAMETERS FOR THIS NOTEBOOK
#
TENANT_ID = " [ YOUR ID HERE ] "
CLIENT_ID = " [ YOUR ID HERE ] "
CLIENT_SECRET = " [ YOUR ID HERE ] "

WORKSPACE_ID = " [ YOUR ID HERE ] "
DATASET_ID = " [ YOUR ID HERE ] "

print("Successfully configured all paramaters for this run.")

StatementMeta(, c5e530bb-9e39-4188-96de-a390e451f2c7, 3, Finished, Available, Finished)

Successfully configured all paramaters for this run.


In [7]:
#
# Import Packages Required for this Notebook
#
import requests
import json
import msal
import time

print("Successfully imported all packages for this notebook.")

StatementMeta(, c5e530bb-9e39-4188-96de-a390e451f2c7, 9, Finished, Available, Finished)

Successfully imported all packages for this notebook.


In [6]:
#
# Connect to Fabric and get the authorization token
#

# Azure AD Details
scope = ["https://analysis.windows.net/powerbi/api/.default"]  # Fabric API scope
authority = f"https://login.microsoftonline.com/{TENANT_ID}"

# Create an MSAL application instance
app = msal.ConfidentialClientApplication(CLIENT_ID, CLIENT_SECRET, authority)

# Get the Access Token
token_response = app.acquire_token_for_client(scopes=scope)

# Check if the token was retrieved successfully
if "access_token" in token_response:
    access_token = token_response["access_token"]
    print("✅ Access Token Retrieved Successfully!")
else:
    print(f"❌ Error getting access token: {token_response.get('error_description')}")

StatementMeta(, c5e530bb-9e39-4188-96de-a390e451f2c7, 8, Finished, Available, Finished)

✅ Access Token Retrieved Successfully!


In [13]:
#
# Refresh the Warehouse semantic model (dataset) in the GOLD layer
#

# API Headers with Access Token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Power BI API URL
url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/datasets/{DATASET_ID}/refreshes"

# Execute the Power BI REST API
response = requests.post(url, headers=headers)
if response.status_code == 202:
    print("✅ Refresh successfully submitted for the Fabric Semantic Model!")
else:
    print("❌ Error:", response.text)

StatementMeta(, c5e530bb-9e39-4188-96de-a390e451f2c7, 15, Finished, Available, Finished)

✅ Refresh successfully submitted for the Fabric Semantic Model!


In [27]:
#
# Check the Warehouse semantic model (dataset) refresh
#

# API Headers with Access Token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Power BI API URL
url = f"https://api.powerbi.com/v1.0/myorg/groups/{WORKSPACE_ID}/datasets/{DATASET_ID}/refreshes"

# Function to check the refresh status and return when done
def check_refresh_status():
    """Polls the API until the dataset refresh is 'Completed' or 'Failed'."""
    while True:
        response = requests.get(url, headers=headers)

        if response.status_code == 202:
            print("🔄 Refresh request accepted. Waiting before polling again...")
            time.sleep(10)
            continue  # Skip the rest and retry

        if response.status_code == 200:
            try:
                response_json = response.json()
                refreshes = response_json.get("value", [])
                if refreshes:
                    latest_refresh = refreshes[0]  # Get the most recent refresh
                    status = latest_refresh.get("status")
                    print(f"🟢 Current status: {status}")

                    if status == "Completed":
                        print("✅ Dataset refresh completed successfully!")
                        break
                    elif status in ["Failed", "Unknown"]:
                        print("❌ Dataset refresh failed or encountered an issue.")
                        break
                else:
                    print("⚠️ No refresh history found.")
                    break
            except json.JSONDecodeError as e:
                print(f"❌ JSON decoding error: {e}. Raw response: {response.text}")
                break
        else:
            print(f"❌ Error fetching refresh status: {response.status_code} - {response.text}")
            break

        time.sleep(10)  # Wait 10 seconds before checking again

# Start polling
check_refresh_status()

StatementMeta(, c5e530bb-9e39-4188-96de-a390e451f2c7, 29, Finished, Available, Finished)

🟢 Current status: Completed
✅ Dataset refresh completed successfully!
